In [13]:
import pandas as pd

### 1.1 - Movielens full - EDA

### a)

In [14]:
movies_df = pd.read_csv('Data/movies.csv')
ratings_df = pd.read_csv('Data/ratings.csv', usecols=['userId', 'movieId', 'rating'])

In [15]:
display(
    movies_df.info(),
    ratings_df.info(),
    movies_df.describe().T,
    ratings_df.describe().T
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  58098 non-null  int64 
 1   title    58098 non-null  object
 2   genres   58098 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753444 entries, 0 to 27753443
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   userId   int64  
 1   movieId  int64  
 2   rating   float64
dtypes: float64(1), int64(2)
memory usage: 635.2 MB


None

None

,count,mean,std,min,25%,50%,75%,max
movieId,58098.0,111919.516197,59862.660956,1.0,72437.75,126549.0,161449.5,193886.0


,count,mean,std,min,25%,50%,75%,max
userId,27753444.0,141942.015571,81707.400091,1.0,71176.0,142022.0,212459.0,283228.0
movieId,27753444.0,18487.999834,35102.625247,1.0,1097.0,2716.0,7150.0,193886.0
rating,27753444.0,3.530445,1.066353,0.5,3.0,3.5,4.0,5.0


### b)

In [16]:
ratings_df

,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
4,1,1449,4.5
...,...,...,...
27753439,283228,8542,4.5
27753440,283228,8712,4.5
27753441,283228,34405,4.5
27753442,283228,44761,4.5
